In [1]:
import numpy as np
from autograd import grad
import seaborn
import autograd.numpy as np
import matplotlib.pyplot as plt
from tqdm import tnrange as trange 

N = 10000
dist = [0.1, 1.0, 10.0]

def p( x, t=1.0 ):
    return np.exp( -10*t*((x-2)**2) ) + 0.3*np.exp( -0.5*10*t*((x+1)**2) )


#metropolis hastings mcmc
def mh_runonce(N, dist, x=0, t=1):
    #give it some random noise
    noise = np.random.normal(0, dist, (N,1))
    noise =  noise[:,0]
    state = []
    count = 0
    #Then test noise for every step
    for t,noisy in enumerate(noise):
        xprime = x + noisy
        stateval = p(xprime)/p(x)
        if(stateval >= 1):
            state.append(xprime)
            x = xprime
            count +=1
        else:
            #if it's not >=1 then x' gets chosen with prob a and x chosen with prob x
            r = min(1, stateval)
            u = uniform(0,1)
            if(r >= u):
                x = xprime
                count += 1
            else:
                x = x
            state.append(x)
    return state, count

#metropolis hastings mcmc


states_0 = []
states_1 = []
states_2 = []
perc0 = 0
perc1 = 0
perc2 = 0
for i in xrange(3):
    state0, percent0 = mh_runonce(N,dist[0])
    states_0.append(state0)
    state1, percent1 = mh_runonce(N,dist[1])
    states_1.append(state1)
    state2, percent2 = mh_runonce(N,dist[2])
    states_2.append(state2)
    perc0 += percent0
    perc1 += percent1
    perc2 += percent2
len(states_0)
        
%matplotlib inline   
%pylab inline
pylab.rcParams['figure.figsize'] = (16.0,5.0)
L = 1000


def plot_lines(states, title):
    index = np.arange(0,L)
    plt.ylim(-3,3)
    for i in xrange(3):
        plt.plot(index, states[i][-1001:-1], linewidth = .5)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("State")
    plt.show()


print "percentage for .1 = ", double(perc0)/(N*3)
print "percentage for 1 = ", double(perc1)/(N*3)
print "percentage for 10 = ", double(perc2)/(N*3)
plot_lines(states_0, "Traces for ss = .1" )
plot_lines(states_1, "Traces for ss = 1" )
plot_lines(states_2, "Traces for ss = 10" )

def plot_hist(state, title):
    plt.ylim(0,1.5)
    bin_wid = .01
    passintop = np.arange(-3,3,bin_wid)
    actualline = p(passintop)
    actualline /= np.sum(actualline*bin_wid)
    plt.hist(state[0]+state[1]+state[2], normed = True, bins = 130)
    plt.plot( passintop, actualline)
    plt.title(title)
    plt.show()

plot_hist(states_0, "Proposal ss = .1")
plot_hist(states_1, "Proposal ss = 1")
plot_hist(states_2, "Proposal ss = 10")


NameError: global name 'uniform' is not defined

In [ ]:
import autograd.numpy as np
from tqdm import tnrange as trange 

def U(q):
    return -np.log(p(q))
# def K(p):
#     return p.T*np.pinv(M)*p/2
# def H(q,p):
#     U(q)+K(p)
# def q(q, p, t, epsilon, grad_U, mi):
#     return q(t) + epsilon(p_leapfrog(p, t, epsilon/2 , grad_U))/mi
# def p_leapfrog(q, p, t, epsilon, grad_U, mi):
#     return p_half_leapfrog(p, t, epsilon, grad_U) - epsilon/2*gradU(q(q, -, t, epsion, grad_U, mi))

# t = 1
# # q = position
# # p = momentum

# # Obtain gradient functions
grad_U = grad( U )      
N = 10000

#hamiltonian mcmc (leapfrogging)
def HMC( U, grad_U, epsilon, L, current_q ):
    m = 1
    sigma = 2.4
    #m = np.diag(marray)
    q = current_q
    p = np.random.normal( q, sigma ) # independent standard normal variates
    current_p = p
    # Make a half step for momentum at the beginning
    p = p - epsilon * grad_U( q ) / 2
    # Alternate full steps for position and momentum
    for i in xrange(1,L):
        # Make a full step for the position
        q = (q + epsilon * p) / m
        # Make a full step for the momentum, except at end of trajectory
        if not(i == L):
            p= p - epsilon * grad_U( q )
    # Make a half step for momentum at the end.
    p= p - epsilon * grad_U( q ) / 2
    # Negate momentum at end of trajectory to make the proposal symmetric
    p = - p
    # Evaluate potential and kinetic energies at start and end of trajectory
    current_U = U( current_q )
    current_K = 1.0/m*sum(p**2)/2
    #current_K =  current_p.T * m * current_p  / 2
    proposed_U = U( q )
    proposed_K = 1.0/m*sum(p**2)/2
    #proposed_K = p.T * pinv.m * p / 2
    # Accept or reject the state at end of trajectory, returning either
    # the position at the end of the trajectory or the initial position
    if (np.random.random_sample(size=1)[0] < np.exp( current_U - proposed_U + current_K - proposed_K )):
        return (q), True # accept
    else:
        return (current_q), False # reject

epsilon = .4
grad_p = grad( p )
holding_1 = []
current_q_1 = 0
count = 0
L = 100

for i in trange(N):
    current_q_1, boolean1 = HMC(U, grad_U, epsilon, L, current_q_1)
    count += boolean1
    holding_1.append(current_q_1)




In [ ]:
print "percentage changed = ", double(count)/N

index = np.arange(0,N)
plt.figure(3)
plt.plot(index, holding_1, linewidth = .5)
plt.show()


bin_wid = .01
passintop = np.arange(-3,3,bin_wid)
actualline = p(passintop)
actualline /= np.sum(actualline*bin_wid)
plt.hist(holding_1, normed = True, bins = 130)
plt.plot( passintop, actualline)
plt.title("Plot Histogram")
plt.show()


Part 3: Observations
You have now coded two different inference algorithms, and a few variants of each. For this section, you must provide a small write-up that compares and contrasts each. 

Answer at least the following questions:
1.What was the acceptance rate of each algorithm? (ie, what percentage of proposals were accepted) 

Answer: (This value is printed on the different algorithms)

2.Why don't some inference algorithms explore both modes of the density?

Answer: Beause the variance wasn't big enough to allow it to reasonably think it would be able to make it into the mode it was in. It was also too small to make it into the other mode.

3.Why do some algorithms stay in the same state repeatedly?

Answer: Because the algorithm rejects the states that don't make sense to it. They reject the ones that are too far out. 

4.Is this good or bad?

Answer: This is bad, we are staying and not exploring the modes at all

5.What were the best values for the variance of the momentum variables and the timestep you found?

Answer: m = 1, I tried lots of values and none worked as well as 1,
Timestep = 100. It works well because at 100 your values are not dependent on those before it. They are also given time to explore and taking samples then. But it's not too much that it goes too far or it takes 50000000 years. 

5.How did you know that they were good?

Answer: Because I tried literally everything else and finally narrowed down on a value that worked well.

All in all I think that doing the Metropolis hastings algorithm was not only easier but it was also more effective and easier to find parameters to fit the curve. sigma = 1 worked significantly better than 10 or .1 for that one though.